<a href="https://colab.research.google.com/github/nduc4nh/CNNResnet_Music_Classification/blob/main/GENRE_CNN_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Genre Data/data10_32.csv')


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
le = preprocessing.LabelEncoder()
genre_encoded = le.fit_transform(df.label)
df['label'] = genre_encoded


In [ ]:
le.inverse_transform(genre_encoded)

array(['catru', 'catru', 'catru', ..., 'rock', 'rock', 'rock'],
      dtype=object)

In [ ]:
import pickle
with open('/content/drive/MyDrive/label_encode','wb') as f:
  pickle.dump(le,f)

In [ ]:
y = df.iloc[:, -1]
df = df.iloc[:, :-1]


In [ ]:
df = df.iloc[:,2:]
df = df.to_numpy()

In [ ]:
df.shape

(37419, 16796)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25)

In [ ]:
del df

In [ ]:
l_valid  = X_train.shape[0]//4
X_valid = X_train[:l_valid,:]
y_valid = y_train[:l_valid]
X_train = X_train[l_valid:,:]
y_train = y_train[l_valid:]

In [ ]:
X_train = X_train.reshape([X_train.shape[0],13,1292])

In [ ]:
X_test = X_test.reshape([X_test.shape[0],13,1292])

In [ ]:
X_valid = X_valid.reshape([X_valid.shape[0],13,1292])

In [ ]:
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]

In [ ]:
import tensorflow.keras as keras

In [ ]:
# model
model = keras.Sequential()

# 1st conv layer
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model.add(keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape))
model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding = 'same'))
model.add(keras.layers.BatchNormalization())

# 2nd conv layer

model.add(keras.layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((3,3), strides=(2,2), padding = 'same'))
model.add(keras.layers.BatchNormalization())
          
# 3rd conv layer

model.add(keras.layers.Conv2D(128, (2,2), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2), padding = 'same'))
model.add(keras.layers.BatchNormalization())

# flatten output and feed it into dense layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
          
# output layer
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 11, 1290, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 645, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 645, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 643, 128)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 322, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 322, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 321, 128)       6

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_valid,y_valid), batch_size=32, epochs=30)

Epoch 1/30
658/658 [==============================] - 47s 23ms/step - loss: 1.8357 - accuracy: 0.4369 - val_loss: 1.3021 - val_accuracy: 0.5780
Epoch 2/30
658/658 [==============================] - 14s 22ms/step - loss: 1.3663 - accuracy: 0.5619 - val_loss: 1.1266 - val_accuracy: 0.6375
Epoch 3/30
658/658 [==============================] - 14s 22ms/step - loss: 1.1834 - accuracy: 0.6124 - val_loss: 1.0401 - val_accuracy: 0.6646
Epoch 4/30
658/658 [==============================] - 14s 22ms/step - loss: 1.0492 - accuracy: 0.6515 - val_loss: 0.9784 - val_accuracy: 0.6813
Epoch 5/30
658/658 [==============================] - 14s 22ms/step - loss: 0.9631 - accuracy: 0.6792 - val_loss: 0.9135 - val_accuracy: 0.6948
Epoch 6/30
658/658 [==============================] - 14s 22ms/step - loss: 0.8688 - accuracy: 0.7085 - val_loss: 0.8813 - val_accuracy: 0.7078
Epoch 7/30
658/658 [==============================] - 15s 22ms/step - loss: 0.7934 - accuracy: 0.7332 - val_loss: 0.8385 - val_accuracy:

In [ ]:
del X_train, X_valid

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

293/293 - 6s - loss: 0.6994 - accuracy: 0.7804

Test accuracy: 0.780438244342804


In [ ]:
model.save("/content/drive/MyDrive/Genre Data/model_h5_2.h5")

In [ ]:
del X_train,X_test,y_train, y_test

In [ ]:
df.iloc[:,0].astype('float64')

0       -549.261047
1       -243.961197
2       -392.869965
3       -429.180481
4       -262.171844
            ...    
29807   -571.208679
29808   -572.879028
29809   -594.335327
29810   -559.491516
29811   -481.592010
Name: 0, Length: 29812, dtype: float64

In [ ]:
df = df.iloc[:,2:]

In [ ]:
def reshape(x,a,b):
    n = x.shape[0]
    re = np.ndarray([0,a,b],buffer = np.array([]))
    while n != 0: 
        idx = x.index
        tmp = x.iloc[0,:].to_numpy().reshape((1,a,b))
        re = np.append(re,tmp,axis = 0)
        x.drop(idx[0],inplace = True)
        n -= 1
    return re

In [ ]:
n = df.shape[0]
p = n//3

In [ ]:
re = np.ndarray([0,13,1292],buffer = np.array([]))


In [ ]:
count = 0

In [ ]:
idx = df.index

In [ ]:
tmp = df.iloc[0,:].to_numpy().reshape((1,13,1292))

In [ ]:
re.shape

(8485, 13, 1292)

In [ ]:
count

9665

In [ ]:
p

9937

In [ ]:
df = df.astype('float32')

In [ ]:
while n != 0: 
  if count == p:
    break
  idx = df.index
  tmp = df.iloc[0,:].to_numpy().reshape((1,13,1292))
  re = np.append(re,tmp,axis = 0)
  n -= 1
  count += 1

KeyboardInterrupt: ignored

In [ ]:
df.iloc[0,:].to_numpy().reshape((1,13,1292))

array([[[-243.9611969 , -258.21350098, -295.59088135, ...,
         -403.86517334, -396.85910034, -406.62399292],
        [ 136.55322266,  138.01483154,  130.82504272, ...,
          167.65782166,  168.28622437,  157.23406982],
        [  57.84633636,   41.29589844,   22.48103333, ...,
           77.99763489,   66.44941711,   60.35290527],
        ...,
        [  -8.4535141 ,   -0.97962987,   10.82798767, ...,
          -25.08483887,  -31.11982346,  -35.54663849],
        [   8.57917786,    5.71994162,    2.95274925, ...,
          -34.7728653 ,  -38.51311493,  -38.10894775],
        [  -7.33879852,   -9.66025066,   -9.87447548, ...,
          -13.65322113,  -12.78897667,  -12.4564724 ]]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
del re

In [ ]:
model = keras.models.load_model(r'/content/drive/MyDrive/model_h5_conv1d_8.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 36750, 64)         256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 36748, 64)         12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12249, 64)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12247, 64)         12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4082, 64)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4080, 128)         24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1360, 128)         0